In [26]:
"Timetable parsing"

from tramnetwork import *

lookup_day = "20250128"

network = TramNetwork()
network.load_day(lookup_day)

Found 203 Tram Stops (20250128).          


In [ ]:
# make animation
from pathlib import Path as FilePath
import shutil, time
import cv2
import os

stop1 = network.search_stops("Schlieren, Geissweid")[0]
stop2 = network.search_stops("Fracht")[0]
start_time = gt_parse_time("12:00:00", lookup_day)
FPS = 10

temp_image_dir = FilePath("_temp_dijkstra_images")
video_name = "dijkstra_animation.mp4"

image_count = 0
image_file_paths = []

temp_image_dir.mkdir(exist_ok=True, parents=True)


def calc_dijkstra_path_to(start_time: datetime, start: TramStop, destination_criterium,
                          start_tram: TramName=None, weight_func=None) -> TramPath:
    global image_count

    visited_stops = set()
    start_connection = TramPath([start], [start_time], [None], [None])
    visit_stack = [(0, start_time, start_connection, start)]

    while len(visit_stack) > 0:
        _, curr_time, prev_stops, stop = visit_stack.pop(0)
        if stop in visited_stops:
            continue
        
        if prev_stops.departure_times[0] is not None:
            img = prev_stops.generate_image(big_path=True, greyscale_background=True)
            img_name = f"frame{image_count:05}.jpg"
            img_path = temp_image_dir / img_name
            image_count += 1
            image_file_paths.append(img_path)

            img.save(img_path)
            print(img_name, end="\r")

        visited_stops.add(stop)
        if destination_criterium(stop):
            return prev_stops

        connections = stop.get_departures_after(curr_time)
        for connection in connections:
            wait_seconds = (connection.arrival_time - curr_time).total_seconds()
            
            if len(prev_stops) >= 2:
                last_tram_name = prev_stops.transportation_names[-2]
            else:
                last_tram_name = start_tram

            if last_tram_name is not None and connection.tram_name != last_tram_name:
                # it's a change of tram!
                if wait_seconds < MIN_CHANGE_BUFFER_SECONDS:
                    continue

            if wait_seconds >= MAX_TRANSITION_SECONDS:
                break

            new_stop = connection.stops[1]
            new_time = connection.arrival_times[1]
            if new_stop in visited_stops:
                continue

            prev_stops_copy = prev_stops.slice(0)
            prev_stops_copy.stop_departure_times[-1] = connection.departure_time
            prev_stops_copy.transportation_names[-1] = connection.tram_name

            prev_stops_copy.add_stop(new_stop, new_time, None, None)
            
            weight = new_time.timestamp()
            if weight_func is not None:
                weight += weight_func(new_stop)

            bisect.insort(visit_stack, (weight, new_time, prev_stops_copy, new_stop))
            
    raise NoConnectionsLeftException("Couldn't find connection in time")

def calc_dijkstra_path_between(start_time: datetime, start: TramStop, destination: TramStop,
                               start_tram: TramName=None, weight_func=None) -> TramPath:
    destination_criterium = lambda stop: stop == destination
    return calc_dijkstra_path_to(start_time, start, destination_criterium, start_tram=start_tram, weight_func=weight_func)

print("Generating images...")
calc_dijkstra_path_between(start_time, stop1, stop2)
print(f"Successfully generated {image_count} images.")

print(f"Writing video: {video_name!r}")

frame = cv2.imread(image_file_paths[0])
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, FPS, (width,height))

for file_path in image_file_paths:
    print(file_path, end="\r")
    video.write(cv2.imread(file_path))

cv2.destroyAllWindows()
video.release()

print(f"Successfully saved video with {FPS=}.")

# remove temp directory again
if temp_image_dir.exists() and temp_image_dir.is_dir():
    shutil.rmtree(temp_image_dir)
print("Successfully removed image directory.")

Generating images...
Successfully generated 202 images.
Writing video: 'djikstra_animation.mp4'
Successfully saved video with FPS=10.
Successfully removed image directory.
